# 🚗 YOLO Detection Server for Kaggle (v2)

This notebook runs mixed YOLO models (v5 and v8/11) on Kaggle with GPU:
1. **Vehicle Detection** - Detect cars, trucks, motorcycles, etc.
2. **Traffic Light Detection** - Detect traffic signals
3. **License Plate Detection + OCR** - Detect and read license plates

## Features
- **Hybrid Model Support**: Runs both YOLOv5 and YOLOv8 models seamlessly
- **Socket.IO**: Real-time connection to Node.js server
- **Ngrok API**: Public REST endpoint

## Setup
1. Enable GPU: Settings → Accelerator → GPU P100
2. Enable Internet: Settings → Internet → On
3. Upload your `.pt` files
4. Run all cells

## 📦 Step 1: Install Dependencies

In [7]:
!pip install ultralytics python-socketio pyngrok flask flask-cors opencv-python-headless pillow -q
!pip install websocket-client python-engineio -q
print("✅ Dependencies installed!")

✅ Dependencies installed!


## ⚙️ Step 2: Configuration

In [8]:
# ============== CONFIGURATION ==============

# Socket.IO Server URL (your Node.js server)
SOCKETIO_SERVER_URL = 'wss://unadjourned-darlene-subcarinated.ngrok-free.dev'  # ⚠️ CHANGE THIS!

# Ngrok Auth Token (for public API URL)
NGROK_AUTHTOKEN = '2pR6eJB7Xm9dbhOwjRHY3o8GzeT_7pSQRkXz6743sRMGnyKPe'

# Model paths
VEHICLE_MODEL_PATH = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-vehicle-best-new.pt'
TRAFFIC_LIGHT_MODEL_PATH = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/mhiot-dentinhieu-best-new.pt'
LP_DETECTOR_MODEL_PATH = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_detector.pt'
LP_OCR_MODEL_PATH = '/kaggle/input/phat-trien-iot-nang-cao/pytorch/default/1/LP_ocr.pt'

# Detection settings
CONFIDENCE_THRESHOLD = 0.5
ENABLE_GPU = True

print("✅ Configuration ready")

✅ Configuration ready


## 🧠 Step 3: Mixed YOLO Model Loader (v5 & v8)

In [9]:
import torch
from ultralytics import YOLO
import os

# Check GPU
device = 'cpu'
if ENABLE_GPU and torch.cuda.is_available():
    device = 'cuda'
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ Using CPU")

def load_yolo_model(path, model_name):
    """Load YOLO model handling both v8 (Ultralytics) and v5 (Torch Hub)"""
    if not os.path.exists(path):
        print(f"⚠️ {model_name} not found at {path}")
        return None, None
    
    try:
        # Try loading as YOLOv8/11 first
        model = YOLO(path)
        # Simple test to check compatibility (some v5 models might load but fail inference)
        # But usually 'TypeError' happens at init if incompatible
        model.to(device)
        print(f"✅ {model_name}: Loaded as YOLOv8/11")
        return model, 'v8'
    except (TypeError, AttributeError, RuntimeError) as e:
        # Fallback to YOLOv5 via torch.hub
        print(f"⚠️ {model_name}: YOLOv8 load failed ({str(e)[:50]}...), trying YOLOv5...")
        try:
            model = torch.hub.load('ultralytics/yolov5', 'custom', path=path, force_reload=True, trust_repo=True)
            model.to(device)
            print(f"✅ {model_name}: Loaded as YOLOv5")
            return model, 'v5'
        except Exception as e2:
            print(f"❌ {model_name}: Failed to load. Error: {e2}")
            return None, None
    except Exception as e:
        print(f"❌ {model_name}: Unexpected error: {e}")
        return None, None

# Load all models
vehicle_model, v_type = load_yolo_model(VEHICLE_MODEL_PATH, "Vehicle Model")
traffic_light_model, tl_type = load_yolo_model(TRAFFIC_LIGHT_MODEL_PATH, "Traffic Light Model")
lp_detector_model, lpd_type = load_yolo_model(LP_DETECTOR_MODEL_PATH, "LP Detector")
lp_ocr_model, lpo_type = load_yolo_model(LP_OCR_MODEL_PATH, "LP OCR")

print("\n🚀 Model loading complete!")

✅ GPU: Tesla P100-PCIE-16GB
✅ Vehicle Model: Loaded as YOLOv8/11
✅ Traffic Light Model: Loaded as YOLOv8/11
✅ LP Detector: Loaded as YOLOv8/11
✅ LP OCR: Loaded as YOLOv8/11

🚀 Model loading complete!


## 🛠️ Step 4: Inference Helper
Normalize outputs from YOLOv5 and YOLOv8.

In [10]:
def run_inference(model, model_type, frame, conf_thres=0.25):
    """Run inference and return standard list of dicts: [{'class', 'conf', 'bbox': [x1,y1,x2,y2,cls_id]}]"""
    results = []
    h, w = frame.shape[:2]
    
    if model is None:
        return results

    try:
        if model_type == 'v8':
            res = model(frame, verbose=False)
            for r in res:
                for box in r.boxes:
                    if float(box.conf[0]) >= conf_thres:
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        results.append({
                            'class': model.names[int(box.cls[0])],
                            'cls_id': int(box.cls[0]),
                            'conf': float(box.conf[0]),
                            'bbox': [x1, y1, x2, y2],
                            'xy_norm': [x1/w, y1/h, x2/w, y2/h]
                        })
        elif model_type == 'v5':
            res = model(frame)
            # YOLOv5 returns (n, 6) tensor: x1, y1, x2, y2, conf, cls
            # Using .xyxy[0] to get tensor
            preds = res.xyxy[0]
            for i in range(len(preds)):
                x1, y1, x2, y2, conf, cls_id = preds[i].tolist()
                if conf >= conf_thres:
                    cls_name = model.names[int(cls_id)] if hasattr(model, 'names') else str(int(cls_id))
                    results.append({
                        'class': cls_name,
                        'cls_id': int(cls_id),
                        'conf': conf,
                        'bbox': [int(x1), int(y1), int(x2), int(y2)],
                        'xy_norm': [x1/w, y1/h, x2/w, y2/h]
                    })
    except Exception as e:
        print(f"Inference error ({model_type}): {e}")
        
    return results

## 🌐 Step 5: Flask API with Ngrok

In [11]:
import cv2
import numpy as np
import base64
import io
import time
import threading
from PIL import Image
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
ngrok_url = None

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'ok',
        'models': {
            'vehicle': v_type,
            'traffic_light': tl_type,
            'lp_detector': lpd_type,
            'lp_ocr': lpo_type
        }
    })

@app.route('/api/detect', methods=['POST'])
def detect():
    try:
        if 'image' in request.files:
            image_bytes = request.files['image'].read()
        elif request.is_json and 'image' in request.get_json():
            image_bytes = base64.b64decode(request.get_json()['image'])
        else:
            return jsonify({'error': 'No image'}), 400
        
        img = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        
        start = time.time()
        all_dets = []
        
        # Run all models
        if vehicle_model: 
            v_res = run_inference(vehicle_model, v_type, frame, CONFIDENCE_THRESHOLD)
            for r in v_res: all_dets.append({'type': 'vehicle', **r})
            
        if traffic_light_model:
            tl_res = run_inference(traffic_light_model, tl_type, frame, CONFIDENCE_THRESHOLD)
            for r in tl_res: all_dets.append({'type': 'traffic_light', **r})
            
        if lp_detector_model:
            lp_res = run_inference(lp_detector_model, lpd_type, frame, 0.3)
            for r in lp_res:
                x1, y1, x2, y2 = r['bbox']
                text = None
                # OCR
                if lp_ocr_model:
                    try:
                        crop = frame[y1:y2, x1:x2]
                        ocr_res = run_inference(lp_ocr_model, lpo_type, crop, 0.5)
                        ocr_res.sort(key=lambda x: x['bbox'][0]) # sort by x1
                        text = ''.join([c['class'] for c in ocr_res])
                    except Exception as e: pass
                
                all_dets.append({'type': 'license_plate', 'text': text, **r})
        
        return jsonify({'detections': all_dets, 'ms': (time.time()-start)*1000})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

if NGROK_AUTHTOKEN:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTHTOKEN)
    tunnel = ngrok.connect(5000, "http")
    print(f"\n🌐 Public URL: {tunnel.public_url}\n")

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.



🌐 Public URL: https://490659f688d0.ngrok-free.app



## 🔌 Step 6: Socket.IO Client
Using the normalized `run_inference` helper.

In [ ]:
import socketio
import queue
import threading
import time
import base64
import io
import cv2
import numpy as np
from PIL import Image
import traceback
running = True
connected = False
camera_queues = {}
camera_threads = {}
# Use logger=False to avoid printing huge base64 image strings
# Fix for YOLOv8/11 Inference Error
def run_inference_v8(model, frame, conf_thres=0.25):
    """Specific inference function for YOLOv8/11 to avoid argument errors"""
    results = []
    h, w = frame.shape[:2]
    try:
        # Run inference properly without extra args that might cause conflicts
        res = model(frame, verbose=False, conf=conf_thres) 
        for r in res:
            for box in r.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                
                class_name = model.names[cls_id] if hasattr(model, 'names') else str(cls_id)
                
                results.append({
                    'class': class_name,
                    'cls_id': cls_id,
                    'conf': conf,
                    'bbox': [x1, y1, x2, y2],
                    'xy_norm': [x1/w, y1/h, x2/w, y2/h]
                })
    except Exception as e:
        print(f"❌ Inference V8 Error: {e}")
    return results
# Update the main processing loop to use this new function for v8 models
# Note: You need to replace the call inside the loop manually or I can update the loop below.
@sio.event
def connect():
    global connected
    connected = True
    print(f"✅ Socket connected to {SOCKETIO_SERVER_URL}")
    sio.emit("join_all_camera")
@sio.event
def disconnect():
    global connected
    connected = False
    print("⚠️ Socket disconnected")
def process_camera(camera_id):
    global running
    print(f"📹 Cam {camera_id} processing started")
    
    while running:
        try:
            # Wait for image
            data = camera_queues[camera_id].get(timeout=1.0) # Timeout 1s to print idle msg
            if data is None: continue
            
            frame, cam_id, img_id, created_at, track_y = data
            
            # --- DEBUG: Print when processing starts ---
            # print(f"  ⚡ Processing frame {img_id[-4:]}...") 
            
            start = time.time()
            all_dets = []
            
            # 1. Vehicle
            if vehicle_model:
                try:
                    # Check model type (assuming v8 for now based on error, or rely on v_type variable if available)
                    # To be safe, try v8 method first
                   res = run_inference_v8(vehicle_model, frame, CONFIDENCE_THRESHOLD)
                   for r in res:
                        all_dets.append({'type': 'vehicle', 'bbox': {'x1': r['xy_norm'][0], 'y1': r['xy_norm'][1], 'x2': r['xy_norm'][2], 'y2': r['xy_norm'][3]}, 'class': r['class'], 'confidence': r['conf']})
                except Exception as e: print(f"❌ Vehicle Model Error: {e}")
            else:
                 # Print only once per second to avoid spam
                 if int(time.time()) % 5 == 0: print("⚠️ Warning: vehicle_model is None")
            # 2. Traffic Light
            if traffic_light_model:
                try:
                    res = run_inference_v8(traffic_light_model, frame, CONFIDENCE_THRESHOLD)
                    for r in res:
                        all_dets.append({'type': 'traffic_light', 'bbox': {'x1': r['xy_norm'][0], 'y1': r['xy_norm'][1], 'x2': r['xy_norm'][2], 'y2': r['xy_norm'][3]}, 'class': r['class'], 'confidence': r['conf']})
                except Exception as e: print(f"❌ Traffic Light Model Error: {e}")
            else:
                 if int(time.time()) % 5 == 0: print("⚠️ Warning: traffic_light_model is None")
            # 3. License Plate
            if lp_detector_model:
                try:
                    res = run_inference(lp_detector_model, lpd_type, frame, 0.3)
                    for r in res:
                        x1, y1, x2, y2 = r['bbox']
                        text = None
                        if lp_ocr_model:
                            try:
                                crop = frame[y1:y2, x1:x2]
                                ocr_res = run_inference(lp_ocr_model, lpo_type, crop, 0.5)
                                ocr_res.sort(key=lambda x: x['bbox'][0])
                                text = ''.join([c['class'] for c in ocr_res])
                            except: pass
                        
                        all_dets.append({'type': 'license_plate', 'text': text, 'bbox': {'x1': r['xy_norm'][0], 'y1': r['xy_norm'][1], 'x2': r['xy_norm'][2], 'y2': r['xy_norm'][3]}, 'confidence': r['conf']})
                except Exception as e: print(f"❌ LP Model Error: {e}")
            inference_ms = (time.time() - start) * 1000
            
            if sio.connected:
                if all_dets:
                    sio.emit('car_detected', {
                        'camera_id': cam_id, 'image_id': img_id, 
                        'detections': all_dets, 'inference_time': inference_ms, 'created_at': created_at
                    })
                    print(f"  ✅ Done: {len(all_dets)} objects ({inference_ms:.0f}ms)")
                else:
                    # Print even if no detections, to confirm it ran
                    print(f"  ✅ Done: 0 objects ({inference_ms:.0f}ms)")
            else:
                 print(f"  ⚠️ Skipped emit (Socket disconnected)")
        except queue.Empty: 
            # print(f"  💤 Cam {camera_id} idle...")
            continue
        except Exception as e:
            print(f"❌ Process loop error: {e}")
            traceback.print_exc()
            time.sleep(0.1)
@sio.on('image')
def on_image(data):
    try:
        # Debug incoming data structure first time
        # print(f"DEBUG: Rx Packet keys: {data.keys()}")
        image_data = data.get('image') or data.get('buffer')
        
        # --- Handle NodeJS Buffer format ---
        if isinstance(image_data, dict) and image_data.get('type') == 'Buffer':
             # print("DEBUG: Converting JSON Buffer array...")
             image_data = bytes(image_data['data'])
        
        if isinstance(image_data, str): 
            image_data = base64.b64decode(image_data)
        
        if not image_data:
            print("❌ Error: Empty image data received")
            return
        try:
            img = Image.open(io.BytesIO(image_data))
            frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"❌ Error decoding image: {e}")
            return
        
        cam_id = data['cameraId']
        if cam_id not in camera_queues:
            camera_queues[cam_id] = queue.Queue(maxsize=10)
            t = threading.Thread(target=process_camera, args=(cam_id,), daemon=True)
            camera_threads[cam_id] = t
            t.start()
            
        try: 
            # Show queue size to detect if GPU is too slow
            q_size = camera_queues[cam_id].qsize()
            if q_size > 5:
                print(f"⚠️ Queue backlog: {q_size}/10")
            
            camera_queues[cam_id].put((frame, cam_id, data['imageId'], data.get('created_at', 0), 0.5), block=False)
        except queue.Full: 
            print("❌ Dropped frame (Queue Full)")
            pass
            
    except Exception as e: 
        print(f"❌ Recv error: {e}")
        traceback.print_exc()
# Connect loop with robust reconnection
def maintain_conn():
    global connected
    while running:
        if not sio.connected:
            connected = False
            try: 
                # Ensure we are clean before connecting
                try: sio.disconnect()
                except: pass
                
                print(f"🔄 Connecting to {SOCKETIO_SERVER_URL}...")
                sio.connect(SOCKETIO_SERVER_URL, transports=['websocket'], wait=False)
            except Exception as e: 
                print(f"❌ Connection failed: {e}")
                time.sleep(5)
        else:
            connected = True
            
        time.sleep(1)
        
threading.Thread(target=maintain_conn, daemon=True).start()
# Keep alive loop
try:
    while running: 
        time.sleep(10)
except: running = False

🔄 Connecting to wss://unadjourned-darlene-subcarinated.ngrok-free.dev...
✅ Socket connected to wss://unadjourned-darlene-subcarinated.ngrok-free.dev
📹 Cam 69589bb7c4bd4ef15b0b713e processing started
Inference error (v8): BaseModel.fuse() got an unexpected keyword argument 'verbose'
  ✅ Done: 0 objects (261ms)


Fusing layers... 
Model summary: 290 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs


⚠️ Queue backlog: 6/10
⚠️ Queue backlog: 7/10
⚠️ Queue backlog: 8/10
⚠️ Queue backlog: 9/10
⚠️ Queue backlog: 10/10
❌ Dropped frame (Queue Full)
⚠️ Queue backlog: 10/10
❌ Dropped frame (Queue Full)
⚠️ Queue backlog: 10/10
❌ Dropped frame (Queue Full)
⚠️ Queue backlog: 10/10
❌ Dropped frame (Queue Full)
Inference error (v8): DetectionModel.forward() got an unexpected keyword argument 'embed'
  ✅ Done: 1 objects (920ms)
Inference error (v8): DetectionModel.forward() got an unexpected keyword argument 'embed'
  ✅ Done: 2 objects (39ms)
⚠️ Queue backlog: 8/10
Inference error (v8): DetectionModel.forward() got an unexpected keyword argument 'embed'
  ✅ Done: 2 objects (42ms)
⚠️ Queue backlog: 8/10
Inference error (v8): DetectionModel.forward() got an unexpected keyword argument 'embed'
  ✅ Done: 2 objects (37ms)
Inference error (v8): DetectionModel.forward() got an unexpected keyword argument 'embed'
  ✅ Done: 2 objects (37ms)
⚠️ Queue backlog: 7/10
Inference error (v8): DetectionModel.forw